In [11]:
"""
## Purpose of script: Visualise the segmentation on images
##
## Author: Yichen He
## Date: 2022/07
## Email: csyichenhe@gmail.com
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import cv2
import sys, os

In [2]:
# Help functions to turn coordinates from the csv file to segmentation mask

def str_to_array(s):
    """
    Cast string array to np array
    
    params:
        s, the array string
    
    return:
        An array of contours [contour_1 , contour_2, ... , contour_n],
        Each contour is an array [x_pt1, y_pt1,x_pt2, y_pt2, ... ,x_ptn, y_ptn]
    
    """
    #Check whether string is '[....] format'
    m = re.match('\[[\S*\s*]*\]', s)
    assert m is not None, "The string is not in \'[....]\'"
    #transer string to array
    if ',' not in s:
        s = re.sub( '\[\s+', '[', s )
        s = re.sub( '\s+\]', ']', s )
        s = re.sub( '\s+', ',', s ).strip()
    result = eval(s)
    if type(result) == tuple:
        result = list(result)
    else:
        result = [result]
    return result



def contour_to_mask(contour_coords, img, scale):
    """
    Convert contours (np arrays) to segmentations (a 2D mask)
    
    params:
        contour_coords, An array of contours [contour_1 , contour_2, ... contour_n]
        img, The image data for the contour, data shape (height x width x channels)
        scale, The scale applied on the mask, the output shape is (height/scale x width/scale)
    
    return:
        A mask where plumage area is 1, non-plumage area is 0.
        The data shape is (height/scale x width/scale)

    """
    
    height,width, _ = img.shape
    mask = np.zeros((height, width))
    for contour_coord in contour_coords:
        mask_temp = np.zeros((height, width))
        outline_coord = [[x//scale,y//scale] for (x,y) in zip(contour_coord[::2], contour_coord[1::2])]
        outline_coord = np.expand_dims(outline_coord , axis = 0)
        cv2.fillPoly(mask_temp, outline_coord, 1)
        mask = np.logical_xor(mask , mask_temp)
    return mask.astype('uint8')


def show_img_with_mask(plt, img, mask):
    """
    Plot image with an overlay segmentation.

    params:
        plt : pyplot library
        img: The image data, shape (height x width x channels)
        Mask: The segmentation data, shape (height x width)
    """
    alpha = 0.5
    
    colour = np.array([255, 182, 193])

    ori_image = img.copy()

    for c in range(3):
        img[:, :, c] = np.where(mask == 1,
                          ori_image[:, :, c] * (1 - alpha) + alpha * colour[c] ,
                          img[:, :, c])

    # if img is not None:
    plt.imshow(img)

In [18]:
# read the annotation file
df = pd.read_csv("data/demo_segmentation_annotations.csv")
# set the image folder
img_folder = "data/img"

output_folder = "data/output"

i=0
# the scale of image (larger scale results smaller images which makes the program runs quicker)
scale= 1
for idx, row in df.iterrows():
# The file name of ith image
    filename = row['file']


    # read the ith images and rescale the image
    img = cv2.imread(img_folder +filename)
    # print("The original image dimension (height, width, channels):" , img.shape)
    img = cv2.resize(img, dsize=(img.shape[1]//scale,img.shape[0]//scale), interpolation=cv2.INTER_CUBIC)
    # print("The rescaled image dimension (height, width, channels):" , img.shape)

    # The string version of ith contours
    contour = row['outline']
    # convert the string contour to array
    contour = str_to_array(contour)
    # print("Number of contours:", len(contour))

    mask = contour_to_mask(contour, img, scale )
    mask = mask*255
   
    cv2.imwrite(os.path.join(output_folder , filename+'.tiff'), mask)
